# Implementation of the MoCo algorithm with training the encoder and after the clasification_head

**The encoder can be pretrained using an unsupervised way. It is after trained using the SimCLR algorithm and the classification head is trained by supervised learning**

In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  # disable GPU devices
os.environ["TFDS_DATA_DIR"] = os.path.expanduser("~/tensorflow_datasets")  # default location of tfds database
os.environ["KERAS_BACKEND"] = "tensorflow"

import keras
from keras import layers, models, regularizers
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

import tensorflow as tf
import tensorflow_datasets as tfds

import json
from pathlib import Path

import matplotlib.pyplot as plt

# Turn off logging for TF
import logging
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "1"
tf.get_logger().setLevel(logging.ERROR)

from dpmhm.datasets import preprocessing, feature, utils, transformer

ds_all, ds_info = tfds.load(
    'CWRU',
    with_info=True,
)

ds0 = ds_all['train']
ds0.element_spec

2024-07-02 13:42:29.259857: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-02 13:42:29.264743: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-02 13:42:29.326754: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-02 13:42:30.598242: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-07-02 13:42:33.191728: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:282] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-07-02 13:42:33.191772: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:134] ret

{'metadata': {'Dataset': TensorSpec(shape=(), dtype=tf.string, name=None),
  'FaultComponent': TensorSpec(shape=(), dtype=tf.string, name=None),
  'FaultLocation': TensorSpec(shape=(), dtype=tf.string, name=None),
  'FaultSize': TensorSpec(shape=(), dtype=tf.float32, name=None),
  'FileName': TensorSpec(shape=(), dtype=tf.string, name=None),
  'LoadForce': TensorSpec(shape=(), dtype=tf.uint32, name=None),
  'NominalRPM': TensorSpec(shape=(), dtype=tf.uint32, name=None),
  'RPM': TensorSpec(shape=(), dtype=tf.uint32, name=None)},
 'sampling_rate': TensorSpec(shape=(), dtype=tf.uint32, name=None),
 'signal': {'BA': TensorSpec(shape=(None,), dtype=tf.float32, name=None),
  'DE': TensorSpec(shape=(None,), dtype=tf.float32, name=None),
  'FE': TensorSpec(shape=(None,), dtype=tf.float32, name=None)}}

Preprocessing on data

In [2]:
outdir = Path('/volatile/home/bm279471/tmp/few_shot_cwru')
os.makedirs(outdir, exist_ok=True)

# compactor = transformer.DatasetCompactor(ds0,
#                                          channels=['DE', 'FE', 'BA'],
#                                          keys=['FaultLocation', 'FaultComponent', 'FaultSize'],
#                                          resampling_rate=12000)

# # Feature extractor
# # Spectrogram is computed on a time window of 0.025 second every 0.0125 second, then converted to decibel scale.
# _func = lambda x, sr: feature.spectral_features(x, sr, 'spectrogram',
# #                                                 n_mfcc=256,
#                                                 time_window=0.025, hop_step=0.0125, n_fft=512,
#                                                 normalize=False, to_db=True)[0]

# extractor = transformer.FeatureExtractor(compactor.dataset, _func)

# # A window of width w correspond to w*0.0125 seconds
# window = transformer.WindowSlider(extractor.dataset, window_size=(64,64), hop_size=(32,32))
# # window = transformer.WindowSlider(extractor.dataset, window_size=(256, 80), hop_size=40)  # 1s, full bandwidth
# # window = transformer.WindowSlider(extractor.dataset, window_size=64, hop_size=32)

# labels = list(compactor.full_label_dict.keys())

# preproc = preprocessing.get_mapping_supervised(labels)
    
# ds_window = window.dataset.map(preproc, num_parallel_calls=tf.data.AUTOTUNE)

# eles = list(ds_window.take(10).as_numpy_iterator())
# input_shape = eles[0][0].shape

# ds_window = ds_window.map(lambda x,y: (tf.ensure_shape(x, input_shape), y), num_parallel_calls=tf.data.AUTOTUNE)

# splits = {'train':0.7, 'transfer':0.1,'val':0.1, 'test':0.1}
# ds_split = utils.split_dataset(ds_window, splits, labels=[i+1 for i in range(len(labels))])

# ds_split['train'].save(str(outdir/'ds_train'))
# ds_split['val'].save(str(outdir/'ds_val'))
# ds_split['transfer'].save(str(outdir/'ds_transfer'))
# ds_split['test'].save(str(outdir/'ds_test'))

# with open(outdir/'lb.json', 'w') as fp:
#     json.dump(labels,fp)

In [3]:
ds_train = tf.data.Dataset.load(str(outdir/'ds_train'))
ds_val = tf.data.Dataset.load(str(outdir/'ds_val'))
ds_transfer = tf.data.Dataset.load(str(outdir/'ds_transfer'))
ds_test = tf.data.Dataset.load(str(outdir/'ds_test'))

with open(outdir/'lb.json', 'r') as fp:
    labels = list(json.load(fp))

Parameters

In [4]:
batch_size = 32
ds_size = utils.get_dataset_size(ds_train) + utils.get_dataset_size(ds_val) +utils.get_dataset_size(ds_transfer) + utils.get_dataset_size(ds_test)
n_embedding  = 128 
kernel_size = (3,3) 
tau = 0.1
projection_dim = 128
momentum_coeff=0.99
queue_size=1000

2024-07-02 13:42:36.600320: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-07-02 13:42:37.105418: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-07-02 13:42:37.596897: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-07-02 13:42:38.041646: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [5]:
ds_train = ds_train.shuffle(ds_size, reshuffle_each_iteration=True).cache().batch(batch_size,drop_remainder=True).prefetch(tf.data.AUTOTUNE)
ds_val = ds_val.batch(batch_size,drop_remainder=True)
ds_transfer=ds_transfer.shuffle(ds_size, reshuffle_each_iteration=True).cache().batch(batch_size,drop_remainder=True).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.batch(1)

eles = list(ds_train.take(1).as_numpy_iterator())
input_shape = eles[0][0][0].shape

2024-07-02 13:42:39.219906: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2024-07-02 13:42:39.225414: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Definition of the encoder

In [6]:
tf.config.run_functions_eagerly(True)

@tf.keras.utils.register_keras_serializable()
class Encoder(models.Model):
    """Convolution Auto-Encoder stacks.

    Notes
    -----
    Shape (H,W) of the input tensor must be power of 2.
    """
    def __init__(self, input_shape, n_embedding, kernel_size, **kwargs):
        super(Encoder, self).__init__(**kwargs)
        self.input_shape = input_shape
        self.n_embedding = n_embedding
        self.kernel_size = kernel_size
        activation = 'relu'
        padding = 'same'
        strides = (2, 2)
        pool_size = (2, 2)
        a_reg = 0.

        # Encoder
        input_enc = layers.Input(shape=input_shape, name='input_enc')
        x = layers.Conv2D(32, kernel_size=kernel_size, activation=activation, padding=padding, name='conv1_enc')(input_enc)
        x = layers.MaxPooling2D(pool_size=pool_size, strides=strides, name='pool1_enc')(x)
        x = layers.BatchNormalization(name='bn1_enc')(x)

        x = layers.Conv2D(64, kernel_size=kernel_size, activation=activation, padding=padding, name='conv2_enc')(x)
        x = layers.MaxPooling2D(pool_size=pool_size, strides=strides, name='pool2_enc')(x)
        x = layers.BatchNormalization(name='bn2_enc')(x)

        x = layers.Conv2D(128, kernel_size=kernel_size, activation=activation, padding=padding, name='conv3_enc')(x)
        x = layers.MaxPooling2D(pool_size=pool_size, strides=strides, name='pool3_enc')(x)
        x = layers.BatchNormalization(name='bn3_enc')(x)

        x = layers.Flatten(name='flatten')(x)
        if a_reg > 0:
            x = layers.Dense(n_embedding, activation=activation, activity_regularizer=regularizers.L1(a_reg), name='fc1_enc')(x)
        else:
            x = layers.Dense(n_embedding, activation=activation, name='fc1_enc')(x)

        self.encoder = models.Model(input_enc, x, name='encoder')

    def call(self, x, training=False):
        return self.encoder(x, training=False)
    
    def get_config(self):
        config = super(Encoder, self).get_config()
        config.update({
            "input_shape": self.input_shape,
            "n_embedding": self.n_embedding,
            "kernel_size": self.kernel_size
        })
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)


encoder = Encoder(input_shape,n_embedding,kernel_size)
config = encoder.get_config()
encoder_clone = Encoder.from_config(config)

Creation of the MoCo model

In [7]:

class MoCo(keras.Model):
    def __init__(self,momentum_coeff,temperature,queue_size):
        super().__init__()

        self.contrastive_augmenter = keras.Sequential([
            layers.RandomFlip("horizontal_and_vertical"),
            layers.RandomZoom(0.2),
            layers.RandomTranslation(height_factor=0.2, width_factor=0.2),
        ], name='Data_augmentation')

        self.encoder = encoder

        self.projection_head = keras.Sequential([
                layers.Dense(projection_dim, activation=None),
            ], name='Projection_head')
        
        self.momentum_coeff = momentum_coeff
        self.temperature = temperature

        # the momentum networks are initialized from their online counterparts
        self.m_encoder = encoder_clone
        self.m_projection_head = keras.models.clone_model(self.projection_head)

        feature_dimensions = self.encoder.encoder.output_shape[1]
        self.feature_queue = tf.Variable(
            tf.math.l2_normalize(
                tf.random.normal(shape=(queue_size, feature_dimensions)), axis=1
            ),
            trainable=False,
        )

    def compile(self, contrastive_optimizer, probe_optimizer, **kwargs):
        super().compile(**kwargs)

        self.contrastive_optimizer = contrastive_optimizer
        self.probe_optimizer = probe_optimizer

        # self.contrastive_loss will be defined as a method
        self.probe_loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

        self.contrastive_accuracy = keras.metrics.SparseCategoricalAccuracy()
        self.probe_accuracy = keras.metrics.SparseCategoricalAccuracy()

    def reset_metrics(self):
        self.contrastive_accuracy.reset_state()
        self.probe_accuracy.reset_state()

    def update_contrastive_accuracy(self, features_1, features_2):
        # self-supervised metric inspired by the SimCLR loss

        # cosine similarity: the dot product of the l2-normalized feature vectors
        features_1 = tf.math.l2_normalize(features_1, axis=1)
        features_2 = tf.math.l2_normalize(features_2, axis=1)
        similarities = tf.matmul(features_1, features_2, transpose_b=True)

        # the similarity between the representations of two augmented views of the
        # same image should be higher than their similarity with other views
        batch_size = tf.shape(features_1)[0]
        contrastive_labels = tf.range(batch_size)
        self.contrastive_accuracy.update_state(
            tf.concat([contrastive_labels, contrastive_labels], axis=0),
            tf.concat([similarities, tf.transpose(similarities)], axis=0),
        )
    
    def cosine_similarity(features_1, features_2):
        features_1 = tf.math.l2_normalize(features_1, axis=1)
        features_2 = tf.math.l2_normalize(features_2, axis=1)
        similarities = tf.matmul(features_1, features_2, transpose_b=True)
        return similarities

    def contrastive_loss(self, query, key):
        # similar to the SimCLR loss, however it uses the momentum networks'
        # representations of the differently augmented views as targets
        query = tf.math.l2_normalize(query, axis=1)
        key = tf.math.l2_normalize(key, axis=1)

        similarities = (
            tf.matmul(
                query,
                tf.concat((key, self.feature_queue), axis=0),
                transpose_b=True,
            )
            / self.temperature
        )

        batch_size = tf.shape(query)[0]
        contrastive_labels = tf.range(batch_size)
        loss = keras.losses.sparse_categorical_crossentropy(
            tf.concat([contrastive_labels], axis=0),
            tf.concat([similarities], axis=0),
            from_logits=True,
        )

        # feature queue update
        self.feature_queue.assign(
            tf.concat(
                [
                    key,
                    self.feature_queue[: -batch_size],
                ],
                axis=0,
            )[:self.feature_queue.shape[0]]
        )
        return tf.reduce_mean(loss)

    @tf.function
    def train_step(self, data):
        train_image, _ = data

        augmented_image = self.contrastive_augmenter(train_image)
        with tf.GradientTape() as tape:
            features = self.encoder(train_image, training=True)
            query = self.projection_head(features)
        
            m_features = self.m_encoder(augmented_image, training=False)
            key = self.m_projection_head(m_features)
            contrastive_loss = self.contrastive_loss(query, key)

        gradients = tape.gradient(
            contrastive_loss,
            self.encoder.trainable_weights + self.projection_head.trainable_weights,)

        self.contrastive_optimizer.apply_gradients(
            zip(gradients,self.encoder.trainable_weights + self.projection_head.trainable_weights,))
        
        self.update_contrastive_accuracy(query, key)

        # the momentum networks are updated by exponential moving average
        for weight, m_weight in zip(self.encoder.weights, self.m_encoder.weights):
            m_weight.assign(
                self.momentum_coeff * m_weight + (1 - self.momentum_coeff) * weight
            )
        for weight, m_weight in zip(
            self.projection_head.weights, self.m_projection_head.weights
        ):
            m_weight.assign(
                self.momentum_coeff * m_weight + (1 - self.momentum_coeff) * weight
            )

        return {
            "c_loss": contrastive_loss,
            "c_acc": self.contrastive_accuracy.result(),
        }

    def test_step(self, data):
        test_image, _ = data
        augmented_image = self.contrastive_augmenter(test_image)
        
        features = self.encoder(test_image, training=False)
        query = self.projection_head(features)
    
        m_features = self.m_encoder(augmented_image, training=False)
        key = self.m_projection_head(m_features)
        self.update_contrastive_accuracy(query, key)

        return {
            "c_acc": self.contrastive_accuracy.result(),
        }

Train the MoCo model

In [8]:
MoCo_model = MoCo(momentum_coeff, tau, queue_size)
     
# MoCo_model.compile(
#     contrastive_optimizer=keras.optimizers.Adam(),
#     probe_optimizer=keras.optimizers.Adam(),
# )

# history = MoCo_model.fit(
#     ds_train.repeat(), 
#     epochs=30, 
#     validation_data=ds_val, 
#     steps_per_epoch=int((0.7*ds_size) // batch_size))

Print evolution of metrics

In [9]:
# fig, ax1 = plt.subplots()

# color = 'tab:red'
# ax1.set_xlabel('Epochs')
# ax1.set_ylabel('c_loss', color=color)
# ax1.plot(history.history['c_loss'], color=color)
# ax1.tick_params(axis='y', labelcolor=color)

# ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

# color = 'tab:blue'
# ax2.set_ylabel('c_acc', color=color) 
# ax2.plot(history.history['val_c_acc'], color=color)
# ax2.tick_params(axis='y', labelcolor=color)

# fig.tight_layout()
# plt.title('Evolution of metrics')
# plt.show()

In [10]:
# MoCo_model.save_weights('moco.weights.h5')
MoCo_model.load_weights('moco.weights.h5')

Define the classification model and train the classification head

In [11]:
# Freeze the encoder weights
MoCo_model.encoder.trainable = False

classification_head = keras.Sequential([
                layers.Dense(128, activation='relu', input_shape=(128,)),
                layers.BatchNormalization(),
                layers.Dense(30) #nb labels
            ], name='Classification_head')

# Create a new classification model
encoder_output = MoCo_model.encoder(MoCo_model.encoder.layers[0].input, training=False)
classification_model = keras.Model(
    inputs=MoCo_model.encoder.layers[0].input,
    outputs=classification_head(encoder_output)
)
classification_model.summary(show_trainable=True, expand_nested=True)

classification_model.compile(
    optimizer=keras.optimizers.Adam(), 
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
    metrics=['accuracy'])

early_stopping = EarlyStopping(
    monitor='val_loss',  
    patience=3,       
    restore_best_weights=True, 
    mode='min'
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.1,         
    patience=2          
)

history = classification_model.fit(
    ds_transfer.repeat(), 
    epochs=60, 
    validation_data=ds_val, 
    steps_per_epoch=int((0.1*ds_size) // batch_size),
    callbacks=[early_stopping, reduce_lr]
)

/volatile/home/bm279471/Documents/.venv/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━┓
┃ Layer (type)                ┃ Output Shape          ┃    Param # ┃ Trai… ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━┩
│ input_enc (InputLayer)      │ (None, 64, 64, 3)     │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ encoder (Encoder)           │ (None, 128)           │  1,142,848 │   N   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └ encoder (Functional)   │ ?                     │  1,142,848 │   N   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│       └ input_enc           │ (None, 64, 64, 3)     │          0 │   -   │
│ (InputLayer)                │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│       └ conv1_enc (Conv2D)  │ (None, 64, 64, 32)    │        896 │   N   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│       └ pool1_enc           │ (None, 32, 32, 32)    │          0 │   -   │
│ (MaxPooling2D)              │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│       └ bn1_enc             │ (None, 32, 32, 32)    │        128 │   N   │
│ (BatchNormalization)        │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│       └ conv2_enc (Conv2D)  │ (None, 32, 32, 64)    │     18,496 │   N   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│       └ pool2_enc           │ (None, 16, 16, 64)    │          0 │   -   │
│ (MaxPooling2D)              │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│       └ bn2_enc             │ (None, 16, 16, 64)    │        256 │   N   │
│ (BatchNormalization)        │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│       └ conv3_enc (Conv2D)  │ (None, 16, 16, 128)   │     73,856 │   N   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│       └ pool3_enc           │ (None, 8, 8, 128)     │          0 │   -   │
│ (MaxPooling2D)              │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│       └ bn3_enc             │ (None, 8, 8, 128)     │        512 │   N   │
│ (BatchNormalization)        │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│       └ flatten (Flatten)   │ (None, 8192)          │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│       └ fc1_enc (Dense)     │ (None, 128)           │  1,048,704 │   N   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ Classification_head         │ (None, 30)            │     20,894 │   Y   │
│ (Sequential)                │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └ dense_1 (Dense)        │ (None, 128)           │     16,512 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └ batch_normalization    │ (None, 128)           │        512 │   Y   │
│ (BatchNormalization)        │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └ dense_2 (Dense)        │ (None, 30)            │      3,870 │   Y   │
└─────────────────────────────┴───────────────────────┴────────────┴───────┘

 Total params: 1,163,742 (4.44 MB)

 Trainable params: 20,638 (80.62 KB)

 Non-trainable params: 1,143,104 (4.36 MB)

Epoch 1/60
50/50 ━━━━━━━━━━━━━━━━━━━━ 14s 263ms/step - accuracy: 0.0910 - loss: 3.5673 - val_accuracy: 0.2894 - val_loss: 2.6015 - learning_rate: 0.0010
Epoch 2/60
50/50 ━━━━━━━━━━━━━━━━━━━━ 17s 344ms/step - accuracy: 0.4044 - loss: 2.1846 - val_accuracy: 0.4775 - val_loss: 1.8850 - learning_rate: 0.0010
Epoch 3/60
50/50 ━━━━━━━━━━━━━━━━━━━━ 17s 341ms/step - accuracy: 0.5719 - loss: 1.6754 - val_accuracy: 0.5869 - val_loss: 1.5223 - learning_rate: 0.0010
Epoch 4/60
50/50 ━━━━━━━━━━━━━━━━━━━━ 17s 350ms/step - accuracy: 0.6719 - loss: 1.3551 - val_accuracy: 0.6913 - val_loss: 1.3036 - learning_rate: 0.0010
Epoch 5/60
50/50 ━━━━━━━━━━━━━━━━━━━━ 18s 360ms/step - accuracy: 0.7437 - loss: 1.1216 - val_accuracy: 0.7412 - val_loss: 1.1551 - learning_rate: 0.0010
Epoch 6/60
50/50 ━━━━━━━━━━━━━━━━━━━━ 17s 338ms/step - accuracy: 0.7905 - loss: 0.9497 - val_accuracy: 0.7694 - val_loss: 1.0136 - learning_rate: 0.0010
Epoch 7/60
50/50 ━━━━━━━━━━━━━━━━━━━━ 18s 368ms/step - accuracy: 0.8274 - loss: 0.

Evaluate the model

In [12]:
evaluation = classification_model.evaluate(ds_test)

print("Evaluation Accuracy: {:.2f}%".format(evaluation[1]*100))

1654/1654 ━━━━━━━━━━━━━━━━━━━━ 43s 26ms/step - accuracy: 0.9265 - loss: 0.2960
Evaluation Accuracy: 93.11%


Fine-tuning

In [13]:
MoCo_model.encoder.trainable = True

classification_model.compile(
    optimizer=keras.optimizers.Adam(1e-4), 
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
    metrics=['accuracy'])

early_stopping = EarlyStopping(
    monitor='val_loss',  
    patience=3,       
    restore_best_weights=True, 
    mode='min'
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.1,         
    patience=2          
)

history = classification_model.fit(
    ds_transfer.repeat(), 
    epochs=40, 
    validation_data=ds_val, 
    steps_per_epoch=int((0.1*ds_size) // batch_size),
    callbacks=[early_stopping, reduce_lr]
)

Epoch 1/40
50/50 ━━━━━━━━━━━━━━━━━━━━ 18s 350ms/step - accuracy: 0.9984 - loss: 0.0234 - val_accuracy: 0.9463 - val_loss: 0.2256 - learning_rate: 1.0000e-04
Epoch 2/40
50/50 ━━━━━━━━━━━━━━━━━━━━ 18s 360ms/step - accuracy: 0.9998 - loss: 0.0197 - val_accuracy: 0.9488 - val_loss: 0.2167 - learning_rate: 1.0000e-04
Epoch 3/40
50/50 ━━━━━━━━━━━━━━━━━━━━ 18s 354ms/step - accuracy: 0.9998 - loss: 0.0178 - val_accuracy: 0.9481 - val_loss: 0.2125 - learning_rate: 1.0000e-04
Epoch 4/40
50/50 ━━━━━━━━━━━━━━━━━━━━ 18s 362ms/step - accuracy: 0.9998 - loss: 0.0164 - val_accuracy: 0.9488 - val_loss: 0.2061 - learning_rate: 1.0000e-04
Epoch 5/40
50/50 ━━━━━━━━━━━━━━━━━━━━ 18s 360ms/step - accuracy: 1.0000 - loss: 0.0148 - val_accuracy: 0.9506 - val_loss: 0.2047 - learning_rate: 1.0000e-04
Epoch 6/40
50/50 ━━━━━━━━━━━━━━━━━━━━ 18s 366ms/step - accuracy: 1.0000 - loss: 0.0136 - val_accuracy: 0.9494 - val_loss: 0.2011 - learning_rate: 1.0000e-04
Epoch 7/40
50/50 ━━━━━━━━━━━━━━━━━━━━ 18s 356ms/step - acc

In [14]:
evaluation = classification_model.evaluate(ds_test)

print("Evaluation Accuracy: {:.2f}%".format(evaluation[1]*100))

   1/1654 ━━━━━━━━━━━━━━━━━━━━ 1:36 58ms/step - accuracy: 1.0000 - loss: 1.6808e-05

1654/1654 ━━━━━━━━━━━━━━━━━━━━ 48s 29ms/step - accuracy: 0.9517 - loss: 0.2194
Evaluation Accuracy: 95.41%
